# Исследование объявлений о продаже квартир

В вашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

### Откройте файл с данными и изучите общую информацию. 

*Изучим первые 10 строк таблицы, случайную выборку, описательную статистику, общую информацию о датасете и построим гистограммы для всех числовых столбцов таблицы.*

In [ ]:
# загрузим библиотеки и прочитаем файл
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')

#выведем первые 10 строк таблицы
data.head(10)

In [ ]:
#выведем на экран случайную строку
data.sample()

In [ ]:
#изучим общую инофрмацию о датасете
data.info()

In [ ]:
#посмотрим на описательную статистику по данным
data.describe()

In [ ]:
#построим гистограммы по числовым значениям данных таблицы
data.hist(figsize=(15, 20))

**Вывод:**

- необходимо заполнить пропущенные значения, где это возможно
- избавится от аномальный значений
- исравить тип данных, где необходимо
- привести названия столбцов к единообразию


### Предобработка данных

**Определеляем пропушенные значения в стоблцах, приведем названия столбцов к единообразию.**

In [ ]:
# переименуем столбы
cols = ['total_images',
        'last_price',
        'total_area_m2',
        'first_day_exposition',
        'number_of_rooms',
        'ceiling_height_m',
        'total_floors_in_building',
        'living_area_m2',
        'floor',
        'is_apartment',
        'is_studio',
        'open_plan',
        'kitchen_area_m2',
        'number_of_balconies',
        'location',
        'nearest_airport_distance_m',
        'city_center_distance_m',
        'parks_numbers_within_3km',
        'nearest_park_distance_m',
        'ponds_number_within_3km',
        'nearest_pond_distance_m',
        'days_exposition']

data.set_axis(cols, axis='columns', inplace=True)

In [ ]:
# убедимся, что столбцы переименованы
data.columns.tolist()

In [ ]:
#отсортируем количество пропущенных значений в столбцах по убыванию
data.isnull().sum().sort_values(ascending=False)

**Заполняем пропуски и меняем тип данных в стобцах.**

In [ ]:
# посмотрим на уникальные значения в столбце 'is_apartment'
data['is_apartment'].unique()

In [ ]:
# nan заменим на False и проверим изменения
# и изменим тип данных на bool

data['is_apartment'] = data['is_apartment'].fillna(False)
data['is_apartment'] = data['is_apartment'].astype(bool)
data['is_apartment'].unique()


In [ ]:
# в столбцах ниже заменим пропущенные значения на 0, так как информация о наличии парков или водоемов 
#не является критически важной для дальнейшего анализа.
# и изменим тип данных на int

data['nearest_park_distance_m'] = data['nearest_park_distance_m'].fillna(0).astype(int)
data['nearest_pond_distance_m'] = data['nearest_pond_distance_m'].fillna(0).astype(int)
data['ponds_number_within_3km'] = data['ponds_number_within_3km'].fillna(0).astype(int)
data['parks_numbers_within_3km'] = data['parks_numbers_within_3km'].fillna(0).astype(int)
  

In [ ]:
# также заменим пропущенные значения на 0 в столбце ниже, предположив, что если нет информации о количестве балконов, 
# значит скорее всего в жилом помещении балконы отсутсвуют.
# и изменим тип данных на int

data['number_of_balconies'] = data['number_of_balconies'].fillna(0).astype(int)


In [ ]:
# заменяем оставшиеся пустые значения высоты потолков на медианное значение по всему датасету
data['ceiling_height_m'] = data.groupby(['location', 'total_floors_in_building'])['ceiling_height_m'].apply(lambda x: x.fillna(x.median()))
data['ceiling_height_m'].fillna(data['ceiling_height_m'].median(), inplace=True)

# заменяем пустые значения жилой площади и площади кухни
living_ratio = data['living_area_m2'].mean() / data['total_area_m2'].mean()
kitchen_ratio = data['kitchen_area_m2'].mean() / data['total_area_m2'].mean()
data['living_area_m2'].fillna(living_ratio * data['total_area_m2'], inplace=True)
data['kitchen_area_m2'].fillna(kitchen_ratio * data['total_area_m2'], inplace=True)


In [ ]:
# удаляем записи, где отсутствует локация и этажность
data.dropna(subset=['location'], inplace=True)
data.dropna(subset=['total_floors_in_building'], inplace=True)

# приводим этажность к целочисленному типу
data['total_floors_in_building'] = data['total_floors_in_building'].astype(int)

In [ ]:
# приводим в целочисленный тип стоимость квартиры
data['last_price'] = data['last_price'].astype(int)

# приводим к типу даты дату публикации
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S')


In [ ]:
# заменим пропуски в days_exposition и заменим тип данных

data['days_exposition'] = data['days_exposition'].fillna(0).astype('int')

In [ ]:
# смотрим количество пропущенных значений 

data.isnull().sum().sort_values(ascending=False)


In [ ]:
#смотрим как изменились типы данных

data.info()

In [ ]:
len(data['location'].unique())

In [ ]:
# поищем дубликаты в столбце 'location'
# приведем всё к нижнему регистру и заменим букву 'ё' на 'е'

data['location'] = data['location'].str.lower()
data['location'] = data['location'].str.replace('ё', 'е', regex=True)
len(data['location'].unique())
#удалили 34 дубликата

In [ ]:
# Удалим строки без города, они будут мешать в анализе и даже пользователям такие объявления не нужны,
# не понятно, где эта квартира.

#data = data.dropna(subset=['location'])
#data['location'].isna().sum()

In [ ]:
data.head()

### Посчитайте и добавьте в таблицу новые столбцы

In [ ]:
# посчитаем цену квадратного метра
data['price_per_square_meter'] = data['last_price'] / data['total_area_m2']

# для удобства просмотра приведем к типу int
data['price_per_square_meter'] = data['price_per_square_meter'].astype('int')


In [ ]:
# добавим столбцы c днем, месяцем и годом

data['day_of_week_exposition'] = data['first_day_exposition'].dt.day_name()
data['month_exposition'] = data['first_day_exposition'].dt.month_name()
data['year_exposition'] = data['first_day_exposition'].dt.year


In [ ]:
# напишем функцию категоризации по этажам, используя метод apply применимо к каждой строке датафрейма через axis = 1

def floor_category(row):
    floors_total = row['total_floors_in_building']
    floor = row['floor']
    if floor == 1:
        return 'первый'
    elif floor == floors_total:
        return 'последний'
    elif 1 < floor < floors_total:
        return 'другой'
    
# категоризуем этажи с помощью функции  
data['floor_category'] = data.apply(floor_category, axis = 1)


In [ ]:
# добавим столбец с расстоянием до центра города в километрах

data['city_center_distance_km'] = data['city_center_distance_m'] / 1000

In [ ]:
# посмотрим на изменения в таблице

data.head()

**Вывод:**

*Добавили столбцы согласно заданию*

### Проведите исследовательский анализ данных

In [ ]:
# изучим данные по общей площади недвижимости

plt.hist(data['total_area_m2'], bins=30, range=(10,300))
plt.title('Площадь квартир')
plt.xlabel('Квадратные метры')
plt.ylabel('Количество квартир')

print('Наибольшие по площади варианты:')
print(data['total_area_m2'].sort_values(ascending=False).head(10))

In [ ]:
data.query('total_area_m2 > 495')

In [ ]:
# изучим данные по жилой площади недвижимости

plt.hist(data['living_area_m2'], bins=30, range=(10,300))
plt.title('Площадь квартир')
plt.xlabel('Квадратные метры')
plt.ylabel('Количество квартир')

print('Наибольшие по площади варианты:')
print(data['living_area_m2'].sort_values(ascending=False).head(10))

In [ ]:
# Работаем с площадью кухни
plt.hist(data['kitchen_area_m2'], bins=30, range=(0,30))
plt.title('Площадь кухни')
plt.xlabel('Квадратные метры')
plt.ylabel('Количество квартир')

print('Наибольшие по площади кухни варианты:')
print(data['kitchen_area_m2'].sort_values(ascending=False).head(10))

In [ ]:
# Работаем со стоимостью
plt.hist(data['last_price'], bins=30, range=(0,20000000))
plt.title('Стоимость квартир')
plt.xlabel('Стоимость')
plt.ylabel('Количество квартир')

print('Наибольшие по цене варианты:')
print(data['last_price'].sort_values(ascending=False).head(10))

In [ ]:
# Работаем с комнатами
plt.hist(data['number_of_rooms'], bins=7, range=(1,7))
plt.title('Комнаты')
plt.xlabel('Количество комнат')
plt.ylabel('Количество квартир')

print('Наибольшие по числу комнат варианты:')
print(data['number_of_rooms'].sort_values(ascending=False).head(10))


In [ ]:
# Работаем с потолками
plt.hist(data['ceiling_height_m'], bins=7)
plt.title('Высота потолков')
plt.xlabel('Метры')
plt.ylabel('Количество квартир')

print('Наибольшие по высоте потолков варианты:')
print(data['ceiling_height_m'].sort_values(ascending=False).head(10))

In [ ]:
# этаж квартиры
plt.hist(data['floor'], bins=100)
plt.title('Высота потолков')
plt.xlabel('Метры')
plt.ylabel('Количество квартир')

print('Наибольшие по высоте потолков варианты:')
print(data['floor'].sort_values(ascending=False).head(10))

In [ ]:
#тип этажа квартиры («первый», «последний», «другой»)

print(data.plot(y='last_price', kind='pie', fisize=(8, 8)))

In [ ]:
#общее количество этажей в доме



In [ ]:
#расстояние до центра города в метрах

In [ ]:
#расстояние до ближайшего аэропорта

In [ ]:
#расстояние до ближайшего парка

In [ ]:
#день и месяц публикации объявления

data.plot(y='day_of_week_exposition', kind='pie')
data.plot(y='month_exposition', kind='pie')

**Вывод

Площадь
Большинство квартир - менее 100 квадратных метров, больше площадь практически не встречается. В подборке есть несколько очень больших и странных вариантов с площадью более 200 квадратных метров.

Стоимость
В основном стоимость квартир - порядка 5 миллионов. Больше 20 миллионов вариантов мало и они выглядят очень странно и некоторые - на порядок дороже.

Комнаты
В основном квартиры состоят из 1, 2 и 3 комнат. Есть небольшая доля 4-комнатных квартир. Варианты, где больше 7 комнат выглядят странно и подозрительно. Даже при объединении нескольких квартир в одну это подозрительно. Возможно, это отдельный дом.

Высота потолков
Большинство вариантов имеют высоту потолков от 2,5 до 3 метров. Больше 4 метров высота потолков выглядит подозрительно. Есть значения в 27 метров (высота 9-этажного дома) - это может быть ошибкой данных и реальная высота 2,7 метра.

Площадь кухни
Самые распространённые кухни - от 5 до 15 метров. Больше 20 метров кухни практически не встречаются. Больше 30 - единичные случаи.

Жилая площадь
Самые распространённые варианты с жилой площадью от 10 до 50 метров. После 80 метров встречаются единичные случаи. Два пика графика вызваны, вероятно, тем, что это жилые площади для 1-комнатной, затем для 2-комнатной и после этого для 3-комнатной квартиры.

Стоимость квадратного метра
Имеет нормальное распределение с пиком в районе 100,000. Варианты дороже 250,000 являются единичными.

In [ ]:
# изучим как быстро продавались квартиры

data.boxplot(['days_exposition'])
plt.ylim(0, 500)
print('Среднее значение:', data['days_exposition'].mean())
print('Медиана:', data['days_exposition'].median())

Какие факторы больше всего влияют на общую (полную) стоимость объекта?
Изучите, зависит ли цена от:
общей площади;
жилой площади;
площади кухни;
количества комнат;
этажа, на котором расположена квартира (первый, последний, другой);
даты размещения (день недели, месяц, год).
Постройте графики, которые покажут зависимость цены от указанных выше параметров. Для подготовки данных перед визуализацией вы можете использовать сводные таблицы.

Посчитайте среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений. Выделите населённые пункты с самой высокой и низкой стоимостью квадратного метра. Эти данные можно найти по имени в столбце locality_name.

Ранее вы посчитали расстояние до центра в километрах. Теперь выделите квартиры в Санкт-Петербурге с помощью столбца locality_name и вычислите среднюю цену каждого километра. Опишите, как стоимость объектов зависит от расстояния до центра города.

### Общий вывод

**Чек-лист готовности проекта**

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл
- [ ]  файлы изучены (выведены первые строки, метод `info()`, гистограммы и т.д.)
- [ ]  определены пропущенные значения
- [ ]  заполнены пропущенные значения там, где это возможно
- [ ]  есть пояснение, какие пропущенные значения обнаружены
- [ ]  изменены типы данных
- [ ]  есть пояснение, в каких столбцах изменены типы и почему
- [ ]  устранены неявные дубликаты в названиях населённых пунктов
- [ ]  устранены редкие и выбивающиеся значения (аномалии) во всех столбцах
- [ ]  посчитано и добавлено в таблицу: цена одного квадратного метра
- [ ]  посчитано и добавлено в таблицу: день публикации объявления (0 - понедельник, 1 - вторник и т.д.)
- [ ]  посчитано и добавлено в таблицу: месяц публикации объявления
- [ ]  посчитано и добавлено в таблицу: год публикации объявления
- [ ]  посчитано и добавлено в таблицу: тип этажа квартиры (значения — «первый», «последний», «другой»)
- [ ]  посчитано и добавлено в таблицу: расстояние в км до центра города
- [ ]  изучены и описаны следующие параметры:
        - общая площадь;
        - жилая площадь;
        - площадь кухни;
        - цена объекта;
        - количество комнат;
        - высота потолков;
        - этаж квартиры;
        - тип этажа квартиры («первый», «последний», «другой»);
        - общее количество этажей в доме;
        - расстояние до центра города в метрах;
        - расстояние до ближайшего аэропорта;
        - расстояние до ближайшего парка;
        - день и месяц публикации объявления
- [ ]  построены гистограммы для каждого параметра
- [ ]  выполнено задание: "Изучите, как быстро продавались квартиры (столбец days_exposition). Этот параметр показывает, сколько дней «висело» каждое объявление.
    - Постройте гистограмму.
    - Посчитайте среднее и медиану.
    - В ячейке типа markdown опишите, сколько обычно занимает продажа. Какие продажи можно считать быстрыми, а какие — необычно долгими?"
- [ ]  выполнено задание: "Какие факторы больше всего влияют на общую (полную) стоимость объекта? Постройте графики, которые покажут зависимость цены от указанных ниже параметров. Для подготовки данных перед визуализацией вы можете использовать сводные таблицы."
        - общей площади;
        - жилой площади;
        - площади кухни;
        - количество комнат;
        - типа этажа, на котором расположена квартира (первый, последний, другой);
        - даты размещения (день недели, месяц, год);
- [ ]  выполнено задание: "Посчитайте среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений. Выделите населённые пункты с самой высокой и низкой стоимостью квадратного метра. Эти данные можно найти по имени в столбце `locality_name`."
- [ ]  выполнено задание: "Ранее вы посчитали расстояние до центра в километрах. Теперь выделите квартиры в Санкт-Петербурге с помощью столбца `locality_name` и вычислите среднюю цену каждого километра. Опишите, как стоимость объектов зависит от расстояния до центра города."
- [ ]  в каждом этапе есть промежуточные выводы
- [ ]  есть общий вывод